In [8]:
import pandas as pd

import tensorflow as tf
import keras 

In [9]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [10]:
print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)



TensorFlow version: 2.11.0
Keras version: 2.11.0


## Load Dataset

In [11]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


11490434/11490434 [==============================] - 0s 0us/step


## Define Model

Sequential is useful for stacking layers where each layer has one input tensor and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the Flatten, Dense, and Dropout layers.

In [12]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])


2023-12-18 12:37:01.904112: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-18 12:37:01.904514: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-18 12:37:01.904624: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-18 12:37:02.887296: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-18 12:37:02.887592: I tensorflow/compile

The model returns a vector of logits or log-odds scores, one for each class.

In [13]:
predictions = model(x_train[:1]).numpy()
predictions

2023-12-18 12:37:43.217724: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:648] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


array([[ 0.68562007, -0.13674214,  0.6263405 , -0.20848517,  0.22122599,
        -0.19586855,  0.11146374,  0.18527396,  0.19700521,  0.01656701]],
      dtype=float32)

The tf.nn.softmax function converts these logits to probabilities for each class: 

In [14]:
tf.nn.softmax(predictions).numpy()


array([[0.16316779, 0.07169471, 0.15377638, 0.06673127, 0.10255354,
        0.06757854, 0.0918928 , 0.09893201, 0.10009945, 0.08357345]],
      dtype=float32)

In [15]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [16]:
loss_fn(y_train[:1], predictions).numpy()

2.694465

## Compile the Model

In [17]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])


## Train the model

In [18]:
model.fit(x_train, y_train, epochs=5)


2023-12-18 12:50:25.750005: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 188160000 exceeds 10% of free system memory.
2023-12-18 12:50:26.189790: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 188160000 exceeds 10% of free system memory.


Epoch 1/5


2023-12-18 12:50:28.467972: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x742bc110 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-18 12:50:28.468062: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Orin, Compute Capability 8.7
2023-12-18 12:50:28.548087: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-18 12:50:28.915491: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-18 12:50:29.032674: I tensorflow/compiler/jit/xla_compilation_cache.cc:480] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 [==============================] - 20s 9ms/step - loss: 0.2996 - accuracy: 0.9130
Epoch 2/5
1875/1875 [==============================] - 16s 8ms/step - loss: 0.1462 - accuracy: 0.9565
Epoch 3/5
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1081 - accuracy: 0.9674
Epoch 4/5
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0888 - accuracy: 0.9725
Epoch 5/5
1875/1875 [==============================] - 18s 9ms/step - loss: 0.0764 - accuracy: 0.9763


## Evaluate the model

In [19]:
model.evaluate(x_test,  y_test, verbose=2)


2023-12-18 12:52:03.638555: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 31360000 exceeds 10% of free system memory.
2023-12-18 12:52:03.713412: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 31360000 exceeds 10% of free system memory.


313/313 - 2s - loss: 0.0740 - accuracy: 0.9769 - 2s/epoch - 6ms/step


[0.0740494355559349, 0.9768999814987183]

In [20]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])


In [22]:
probability_model(x_test[:1])


<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[5.8921142e-08, 3.2722447e-08, 4.4597273e-06, 9.0920628e-05,
        8.3626550e-11, 2.0013824e-06, 1.1536566e-12, 9.9989533e-01,
        9.8892303e-07, 6.1965329e-06]], dtype=float32)>